In [1]:
import pandas as pd
import numpy as np

import sklearn
from scipy import stats

from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
#Класс для импутирования по группированию

class ImputeGroupBy(BaseEstimator, TransformerMixin):
    """
    Параметры:
    method_impute_num: метод импутации количественных признаков mean|median|mode
    group_by_col: по какому предиктору делаем группировку
    copy: создавать ли копию входного датафрейма
    """
    def __init__(self, group_by_col, method_impute_num="mean", copy=True):
        self.method_impute_num = method_impute_num
        self.group_by_col = group_by_col
        self.copy = copy
    
    def fit(self, X, y=None):
        # Создаём копию, чтобы не менять исходный датафрейм
        if self.copy:
            X = X.copy()

        # Словарь преобразований
        self._encoder_dict = {}

        # Проверяем тип данных входного массива. 
        # Если не DataFrame, то преобразуем в него,
        # Заменяя None и nan на np.nan
        if isinstance(X, pd.DataFrame) == False:
            X = pd.DataFrame(X)
            X = X.astype(object).replace("None", np.nan)
            X = X.astype(object).replace("nan", np.nan)
            
        # Цикл идёт по всем предикторам, кроме предиктора, выбранного для группировки. 
        # На первом шаге цикла проверяется, какой тип у предиктора
        # Если object, то идет группировка по моде. 
        # Если иной, то группировка в зависимости от заданного параметра method_impute_num
        for col in X.loc[:, X.columns != self.group_by_col].columns:
            if X[col].dtype == "object":
                self.dict_col = X.groupby(self.group_by_col)[col].\
                    agg(lambda x: next(iter(x.mode()), None)).to_frame().to_dict()
                self._encoder_dict.update(self.dict_col)
            else:
                if self.method_impute_num == "mean":
                    self.dict_col = X.groupby(self.group_by_col)[col].\
                        agg(lambda x: x.mean()).to_frame().to_dict()
                    self._encoder_dict.update(self.dict_col)
                elif self.method_impute_num == "median":
                    self.dict_col = X.groupby(self.group_by_col)[col].\
                        agg(lambda x: x.median()).to_frame().to_dict()
                    self._encoder_dict.update(self.dict_col)
                elif self.method_impute_num == "mode":
                    self.dict_col = X.groupby(self.group_by_col)[col].\
                        agg(lambda x: next(iter(x.mode()), None)).to_frame().to_dict()
                    self._encoder_dict.update(self.dict_col)
        return self

    def transform(self, X):
        # Создаём копию, чтобы не менять исходный датафрейм
        if self.copy:
            X = X.copy()

        # Проверяем тип данных входного массива. 
        # Если не DataFrame, то преобразуем в него,
        # Заменяя None и nan на np.nan
        if isinstance(X, pd.DataFrame) == False:
            X = pd.DataFrame(X)
            X = X.astype(object).replace("None", np.nan)
            X = X.astype(object).replace("nan", np.nan)
        
        # Цикл идёт по всем предикторам, кроме предиктора, выбранного для группировки.
        # Замена пропусков согласно атрибуту класса (словарю) _encoder_dict из метода fit
        for col in X.loc[:, X.columns != self.group_by_col].columns:
            X[col] = X[col].fillna(X[self.group_by_col].map(self._encoder_dict[col]))
        return X

In [3]:
df_train = pd.DataFrame([[5, None, 23],
                        [17, "v", 30],
                        [10, "v", 3],
                        [10, "z", 22],
                        [None, "a", 23],
                        [2, "a", None]],
                        columns=['A', 'B', 'C'])
df_train

,A,B,C
0,5.0,None,23.0
1,17.0,v,30.0
2,10.0,v,3.0
3,10.0,z,22.0
4,NaN,a,23.0
5,2.0,a,NaN


In [4]:
df_test = pd.DataFrame([[5, "x", None],
                        [None, "v", 30],
                        [16, "v", 3],
                        [16, None, 22],
                        [15, "a", 22],
                        [2, "a", 3]],
                        columns=['A', 'B', 'C'])
df_test

,A,B,C
0,5.0,x,NaN
1,NaN,v,30.0
2,16.0,v,3.0
3,16.0,None,22.0
4,15.0,a,22.0
5,2.0,a,3.0


In [9]:
%time

ImputeGroupBy("C", method_impute_num="mean").fit(df_train).transform(df_test)

Wall time: 0 ns


,A,B,C
0,5.0,x,NaN
1,17.0,v,30.0
2,16.0,v,3.0
3,16.0,z,22.0
4,15.0,a,22.0
5,2.0,a,3.0


In [6]:
nmp_1 = np.array([[3, None, 3], 
                  [6, "v", 2], 
                  [10, None, 3], 
                  [5, "z", 23], 
                  [17, None, 3], 
                  [None, "j", 23]])
df = nmp_1
df

array([[3, None, 3],
       [6, 'v', 2],
       [10, None, 3],
       [5, 'z', 23],
       [17, None, 3],
       [None, 'j', 23]], dtype=object)

In [7]:
nmp_2 = np.array([[10, "v", 3], 
                  [17, "v", 3], 
                  [9, "l", 23], 
                  [10, "z", 3], 
                  [17, np.nan, 3], 
                  [np.nan, "j", 23]])
df = nmp_2
df

array([['10', 'v', '3'],
       ['17', 'v', '3'],
       ['9', 'l', '23'],
       ['10', 'z', '3'],
       ['17', 'nan', '3'],
       ['nan', 'j', '23']], dtype='<U11')